In [2]:
import langchain
from langchain import OpenAI, LLMChain, PromptTemplate
import textwrap
import simple_llm_cache

verbose = True
temperature = 0.5

langchain.llm_cache = simple_llm_cache.SimpleLlmCache(verbose)

def dump_cache_stats_since_last_call():
    langchain.llm_cache.dump_cache_stats()
    langchain.llm_cache.clear_cache_stats()
    
template = """Answer the following question as if you are a {character} character:
{question}
"""

prompt = prompt = PromptTemplate(
    input_variables=["character", "question"],
    template=template)

llm = OpenAI(
    temperature=temperature,
    model_name = "gpt-4") # see https://platform.openai.com/docs/models/gpt-4

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=verbose)

output = chain.predict(
    character="pirate",
    question="What is a neural network?")
print(textwrap.fill(output, width=80))

dump_cache_stats_since_last_call()



> Entering new LLMChain chain...
Prompt after formatting:
Answer the following question as if you are a pirate character:
What is a neural network?

Cache hit for key "Answer the following question as if you are a pirate character:\nWhat is a neural network?\n ::: [('_type', 'openai-chat'), ('model_name', 'gpt-4'), ('stop', None), ('temperature', 0.5)])"

> Finished chain.
Arr matey, a neural network be like a seafarin' crew, workin' together to make
sense of the world. It be a system of algorithms, much like a maze of secret
maps, that be designed to recognize patterns. They interpret sensory data
through a kind of machine perception, labelin' or clusterin' raw input. They be
able to recognize numerical patterns, even if they be hidden in heaps of data,
just like findin' hidden treasure in the vast ocean, arr!
LLM Cache: 1 hits, 0 misses, 0 stores
